In [4]:
import sys;
import os;
import glob;
import math;
import numpy as np;
import glob;
import random;
import time;
import torch;
import torch.optim as optim;
import torch.nn as nn;

sys.path.append(os.getcwd());
sys.path.append(os.path.abspath('../../'));
sys.path.append(os.path.abspath('../../../'));
# sys.path.append(os.path.join(os.getcwd(), 'torch/resources'));
import common.utils as U;
import common.opts as opts;
# import resources.models as models;
import th.resources.calculator as calc;
import common.tlopts as tlopts
# import resources.train_generator as train_generator;
import argparse
from itertools import repeat

In [5]:
import th.resources.models as models

In [6]:
from SharedLibs.datestring import genDataTimeStr, getDateStr

In [7]:
#Reproducibility
seed = 42;
random.seed(seed);
np.random.seed(seed);
torch.manual_seed(seed);
if torch.cuda.is_available():
    torch.cuda.manual_seed(seed);
if torch.backends.mps.is_available():
    torch.mps.manual_seed(seed)
torch.backends.cudnn.deterministic = True;
torch.backends.cudnn.benchmark = False;

In [9]:
second_pruned_model = "../../../trained_models/step_4_second_stage_pruning/multifold/uec_4C_IterPrun_fold5_ratio85.0_for_test.pt"

## define the variables opt and set values

In [10]:
def getOpts():
    parser = argparse.ArgumentParser(description='Transfer Learning for ACDNet');
    parser.add_argument('--netType', default='ACDNetV2',  required=False);
    parser.add_argument('--data', default='trainSet',  required=False);
    parser.add_argument('--dataset', required=False, default='uec_iot', choices=['2']);
    parser.add_argument('--BC', default=True, action='store_true', help='BC learning');
    parser.add_argument('--strongAugment', default=True,  action='store_true', help='Add scale and gain augmentation');
    #在ipynb中，不能使用parser.parse，要改用parser.parse_known_args()
    opt, unknown = parser.parse_known_args();
    return opt
    # opt = parser.parse_args();

## Define the Generator

In [11]:
class TLGenerator():
    #Generates data for Keras
    def __init__(self, samples=None, labels=None, options=None, classes_dict=None):
        random.seed(42);
        #Initialization
        print(f"length of samples:{len(samples)}")
        self.data = [(samples[i], labels[i]) for i in range (0, len(samples))];
        self.opt = options;
        self.batch_size = options.batchSize;
        self.preprocess_funcs = self.preprocess_setup();
        self.mapdict = classes_dict

    def __len__(self):
        #Denotes the number of batches per epoch
        return int(np.floor(len(self.data) / self.batch_size));
        #return len(self.samples);

    def __getitem__(self, batchIndex):
        #Generate one batch of data
        batchX, batchY = self.generate_batch_select_fixed_class(batchIndex);
        batchX = np.expand_dims(batchX, axis=1);
        batchX = np.expand_dims(batchX, axis=3);
        return batchX, batchY

    def generate_batch(self, batchIndex):
        #Generates data containing batch_size samples
        sounds = [];
        labels = [];
        indexes = None;
        for i in range(self.batch_size):
            # Training phase of BC learning
            # Select two training examples
            while True:
                sound1, label1 = self.data[random.randint(0, len(self.data) - 1)]
                sound2, label2 = self.data[random.randint(0, len(self.data) - 1)]
                if label1 != label2:
                    break
            sound1 = self.preprocess(sound1)
            sound2 = self.preprocess(sound2)

            # Mix two examples
            r = np.array(random.random())
            sound = U.mix(sound1, sound2, r, self.opt.sr).astype(np.float32)
            # print(f"sound length after U.mix is {len(sound)}")
            # print(f"nClasses:{self.opt.nClasses}, type of mapdict:{type(self.mapdict)}, type of label1:{type(label1)}")
            eye = np.eye(self.opt.nClasses)
            idx1 = self.mapdict[str(label1)]- 1
            idx2 = self.mapdict[str(label2)] - 1
            label = (eye[idx1] * r + eye[idx2] * (1 - r)).astype(np.float32)
            # label = (eye[label1] * r + eye[label2] * (1 - r)).astype(np.float32)

            #For stronger augmentation
            sound = U.random_gain(6)(sound).astype(np.float32)
            # print(f"sound length after U.random_gain is {len(sound)}")
            sounds.append(sound);
            labels.append(label);

        sounds = np.asarray(sounds);
        labels = np.asarray(labels);
        # print(f"batchIndex is {batchIndex}, total sounds is {len(sounds)}")
        # print(f"labels in generate_batch is:\n{labels}")

        return sounds, labels;

    def generate_batch_select_fixed_class(self, batchIndex):
        #Generates data containing batch_size samples
        sounds = [];
        labels = [];
        indexes = None;
        #two variables recording alarm and moaning sounds count
        alarm_selected = 0;
        moaning_selected = 0;
        help_eng_selected = 0;
        for i in range(self.batch_size):
            # Training phase of BC learning
            # Select two training examples
            while True:
                # print("enter while true")
                sound1, label1 = self.data[random.randint(0, len(self.data) - 1)]
                sound2, label2 = self.data[random.randint(0, len(self.data) - 1)]
                lbl1_int = np.int16(label1);
                lbl2_int = np.int16(label2);
                if (lbl1_int == 52 and lbl2_int == 99) or (lbl1_int == 99 and lbl2_int ==52):
                    if (alarm_selected == moaning_selected) and (alarm_selected == help_eng_selected):
                        alarm_selected += 1;
                        break;
                if (lbl1_int == 56 and lbl2_int == 99) or (lbl1_int == 99 and lbl2_int == 56):
                    if (moaning_selected < alarm_selected) and (moaning_selected == help_eng_selected):
                        moaning_selected += 1;
                        break;
                if (lbl1_int == 71 and lbl2_int == 99) or (lbl1_int == 99 and lbl2_int == 71):
                    if (help_eng_selected < alarm_selected) and (help_eng_selected < moaning_selected):
                        help_eng_selected += 1;
                        break;
            # print(f"escape for loop");
            sound1 = self.preprocess(sound1)
            sound2 = self.preprocess(sound2)

            # Mix two examples
            r = np.array(random.random());
           
            # print(f"r:{r}, lbl1:{label1}, lbl2:{label2}")  
            sound = U.mix(sound1, sound2, r, self.opt.sr).astype(np.float32)
            eye = np.eye(self.opt.nClasses)
            idx1 = self.mapdict[str(label1)]- 1
            idx2 = self.mapdict[str(label2)] - 1
            label = (eye[idx1] * r + eye[idx2] * (1 - r)).astype(np.float32)
            

            #For stronger augmentation
            sound = U.random_gain(6)(sound).astype(np.float32)
            # print(f"sound length after U.random_gain is {len(sound)}")
            sounds.append(sound);
            labels.append(label);

        sounds = np.asarray(sounds);
        labels = np.asarray(labels);
        
        return sounds, labels;

    def preprocess_setup(self):
        funcs = []
        if self.opt.strongAugment:
            funcs += [U.random_scale(1.25)]

        funcs += [U.padding(self.opt.inputLength // 2),
                  U.random_crop(self.opt.inputLength),
                  U.normalize(32768.0)]
        return funcs

    def preprocess(self, sound):
        for f in self.preprocess_funcs:
            sound = f(sound)

        return sound;

## Define Trainer

In [29]:
class ReTrainer:
    def __init__(self, opt=None, classes_dict=None):
        self.opt = opt;
        self.testX = None;
        self.testY = None;
        self.bestAcc = 0.0;
        self.bestAccEpoch = 0;
        self.trainGen = getTrainGen(opt,classes_dict=classes_dict)#train_generator.setup(opt, split);
        # self.opt.trainer = self;
        # self.trainGen = getTrainGen(self.opt, self.opt.splits)#train_generator.setup(self.opt, self.opt.split);
        # self.pretrainedmodelpath = "./resources/pretrained_models/acdnet20_20khz_fold4.h5"

    def Train(self):
        train_start_time = time.time();
        state = torch.load(second_pruned_model, map_location=self.opt.device)
        weights = state['weight']
        config = state['config']
        print(f"config is {config}")
        net = models.GetACDNetModel(input_len=20150, sr=20000, nclass=self.opt.nClasses, channel_config=config).to(self.opt.device);
        net.load_state_dict(weights);
         #show the acdnet structures
        # net = getPrunedModel(pruned_model_path=pruned_acdnet)
        #print networks parameters' require_grade value
        # for k_, v_ in net.named_parameters():
        #     print(f"{k_}:{v_.requires_grad}")
        print('ACDNet model has been prepared for training');

        calc.summary(net, (1,1,self.opt.inputLength));
        # net = net.cuda();
        # training_text = "Re-Training" if self.opt.retrain else "Training from Scratch";
        # print("{} has been started. You will see update after finishing every training epoch and validation".format(training_text));

        lossFunc = torch.nn.KLDivLoss(reduction='batchmean');
        optimizer = optim.SGD(net.parameters(), lr=self.opt.LR, weight_decay=self.opt.weightDecay, momentum=self.opt.momentum, nesterov=True);

        # self.opt.nEpochs = 1957 if self.opt.split == 4 else 2000;
        for epochIdx in range(self.opt.nEpochs):
            epoch_start_time = time.time();
            optimizer.param_groups[0]['lr'] = self.__get_lr(epochIdx+1);
            cur_lr = optimizer.param_groups[0]['lr'];
            running_loss = 0.0;
            running_acc = 0.0;
            n_batches = math.ceil(len(self.trainGen.data)/self.opt.batchSize);
            for batchIdx in range(n_batches):
                # with torch.no_grad():
                x,y = self.trainGen.__getitem__(batchIdx)
                x = torch.tensor(np.moveaxis(x, 3, 1)).to(self.opt.device);
                y = torch.tensor(y).to(self.opt.device);
                # zero the parameter gradients
                optimizer.zero_grad();

                # forward + backward + optimize
                # outputs = net(x);#in office and use cpu
                x = x.type(torch.cuda.FloatTensor) #use apple m2
                outputs = net(x)
               
                res_y = y.argmax(dim=1)
                res_y = res_y.type(torch.cuda.FloatTensor)
                running_acc += ((( outputs.data.argmax(dim=1) == res_y)*1).float().mean()).item();
                y = y.type(torch.cuda.FloatTensor)
                
                loss = lossFunc(outputs.log(), y);
                loss.backward();
                optimizer.step();

                running_loss += loss.item();

            tr_acc = (running_acc / n_batches)*100;
            tr_loss = running_loss / n_batches;

            #Epoch wise validation Validation
            epoch_train_time = time.time() - epoch_start_time;

            net.eval();
            val_acc, val_loss = self.__validate(net, lossFunc);
            #Save best model
            # self.__save_model(val_acc, epochIdx, net);
            # ratio, acc, tr_acc, epochIdx, net
            self.__do_save_model(self.opt.pruningRatio*100, val_acc, tr_acc, epochIdx, net);
            self.__on_epoch_end(epoch_start_time, epoch_train_time, epochIdx, cur_lr, tr_loss, tr_acc, val_loss, val_acc);

            running_loss = 0;
            running_acc = 0;
            net.train();

        total_time_taken = time.time() - train_start_time;
        print("Execution finished in: {}".format(U.to_hms(total_time_taken)));

    def load_test_data(self):
        # data = np.load(os.path.join(self.opt.data, self.opt.dataset, 'test_data_{}khz/fold{}_test4000.npz'.format(self.opt.sr//1000, self.opt.split)), allow_pickle=True);
        data = np.load(self.opt.valData, allow_pickle=True);
        print(f"device is :{self.opt.device}")
        print(f"len of Y:{len(data['y'])}")
        # self.testX = torch.tensor(np.moveaxis(data['x'], 3, 1)).to(self.opt.device);
        dataX = np.moveaxis(data['x'], 3, 1).astype(np.float32);
        self.testX = torch.tensor(dataX).to(self.opt.device);
        self.testY = torch.tensor(data['y']).type(torch.float32).to(self.opt.device);

    def __get_lr(self, epoch):
        divide_epoch = np.array([self.opt.nEpochs * i for i in self.opt.schedule]);
        decay = sum(epoch > divide_epoch);
        if epoch <= self.opt.warmup:
            decay = 1;
        return self.opt.LR * np.power(0.1, decay);

    def __get_batch(self, index):
        x = self.trainX[index*self.opt.batchSize : (index+1)*self.opt.batchSize];
        y = self.trainY[index*self.opt.batchSize : (index+1)*self.opt.batchSize];
        return x.to(self.opt.device), y.to(self.opt.device);

    def __validate(self, net, lossFunc):
        if self.testX is None:
            self.load_test_data();
        net.eval();
        with torch.no_grad():
            y_pred = None;
            batch_size = len(self.testX);#(self.opt.batchSize//self.opt.nCrops)*self.opt.nCrops;
#             for idx in range(math.ceil(len(self.testX)/batch_size)):
#             for idx in range(len(self.testX)):
#             x = self.testX[idx*batch_size : (idx+1)*batch_size];
            x = self.testX[:];
            #please refer: https://stackoverflow.com/questions/55266154/pytorch-preferred-way-to-copy-a-tensor
            x = x.clone().detach();#torch.tensor(x)
            x = x.type(torch.cuda.FloatTensor) # use apple cuda
            scores = net(x);
            y_pred = scores.data if y_pred is None else torch.cat((y_pred, scores.data));
            acc, loss = self.__compute_accuracy(y_pred, self.testY, lossFunc);
        net.train();
        return acc, loss;

    #Calculating average prediction (10 crops) and final accuracy
    def __compute_accuracy(self, y_pred, y_target, lossFunc):
        # print(f"shape of y_pred:{y_pred.shape}");
        # print(f"shape of y_target:{y_target.shape}");
        with torch.no_grad():
            #Reshape to shape theme like each sample comtains 10 samples, calculate mean and find theindices that has highest average value for each sample
            if self.opt.nCrops == 1:
                y_pred = y_pred.argmax(dim=1);
                y_target = y_target.argmax(dim=1);
            else:
                y_pred = (y_pred.reshape(y_pred.shape[0]//self.opt.nCrops, self.opt.nCrops, y_pred.shape[1])).mean(dim=1).argmax(dim=1);
                y_target = (y_target.reshape(y_target.shape[0]//self.opt.nCrops, self.opt.nCrops, y_target.shape[1])).mean(dim=1).argmax(dim=1);
                print(f"after: len of y_pred:{len(y_pred)}, len of y_target:{len(y_target)}")
            y_target = y_target.cuda()
            acc = (((y_pred==y_target)*1).float().mean()*100).item();
            # valLossFunc = torch.nn.KLDivLoss();
            loss = lossFunc(y_pred.float().log(), y_target.float()).item();
            # loss = 0.0;
        return acc, loss;

    def __on_epoch_end(self, start_time, train_time, epochIdx, lr, tr_loss, tr_acc, val_loss, val_acc):
        epoch_time = time.time() - start_time;
        val_time = epoch_time - train_time;
        line = 'SP-{} Epoch: {}/{} | Time: {} (Train {}  Val {}) | Train: LR {}  Loss {:.2f}  Acc {:.2f}% | Val: Loss {:.2f}  Acc(top1) {:.2f}% | HA {:.2f}@{}\n'.format(
            self.opt.splits, epochIdx+1, self.opt.nEpochs, U.to_hms(epoch_time), U.to_hms(train_time), U.to_hms(val_time),
            lr, tr_loss, tr_acc, val_loss, val_acc, self.bestAcc, self.bestAccEpoch);
        # print(line)
        sys.stdout.write(line);
        sys.stdout.flush();

    # def __save_model(self, acc, epochIdx, net):
    #     if acc > self.bestAcc:
    #         self.bestAcc = acc;
    #         self.bestAccEpoch = epochIdx +1;
    #         save_model_name = self.opt.model_name.format(self.opt.pruningRatio*100,acc, epochIdx, genDataTimeStr());
    #         save_model_fullpath = self.opt.saveDir + save_model_name;
    #         print(f"save model to {save_model_fullpath}")
    #         torch.save({'weight':net.state_dict(), 'config':net.ch_config}, save_model_fullpath);

    def __save_model_v2(self, acc, train_acc, epochIdx, net):
        if acc > self.bestAcc and acc > self.opt.first_save_acc:
            self.bestAcc = acc;
            self.bestAccEpoch = epochIdx +1;
            self.__do_save_model(self.opt.pruningRatio*100, acc, train_acc, epochIdx, net);
        else:
            if acc > self.opt.save_val_acc and train_acc > self.opt.save_train_acc: 
                self.__do_save_model(self.opt.pruningRatio*100, acc, train_acc, epochIdx, net);
            else:
                pass

    def __do_save_model(self, ratio, acc, tr_acc, epochIdx, net):
        save_model_name = self.opt.model_name.format(ratio, acc, tr_acc, epochIdx+1, genDataTimeStr());
        save_model_fullpath = self.opt.saveDir + save_model_name;
        print(f"save model to {save_model_fullpath}")
        torch.save({'weight':net.state_dict(), 'config':net.ch_config}, save_model_fullpath);
        # logObj.write(f"save model:{self.opt.model_name}, bestAcc:{self.bestAcc}, valAcc:{acc}, trainAcc:{tr_acc}, record@{epochIdx}-epoch");
        # logObj.write("\n");
        # logObj.flush();

In [30]:
def getTrainGen(opt=None, split=None, classes_dict=None):
    dataset = np.load(opt.trainData, allow_pickle=True);
    train_sounds = []
    train_labels = []
    train_sounds = dataset['{}'.format(opt.current_fold)].item()['sounds']
    train_labels = dataset['{}'.format(opt.current_fold)].item()['labels']
    trainGen = TLGenerator(train_sounds, train_labels, opt, classes_dict=classes_dict);
    trainGen.preprocess_setup();
    return trainGen

In [31]:

""" recording training settings and results
===================================
weight-pruning-ratio:
final acc:93.03
epoch:539
opt.batchSize = 32;
opt.LR = 0.05;
opt.weightDecay = 5e-3;
opt.momentum = 0.04;
opt.nEpochs = 1000;
opt.schedule = [0.15, 0.3, 0.45];
opt.warmup = 10;
====================================================================
###針對二次剪枝後的重訓練，這個設定滿有效果的，要實驗是哪個參數造成的效果。
epoch:293
acc:89.4
tr_acc:76.4
opt.batchSize = 64;
opt.LR = 0.1;
opt.weightDecay = 5e-3;
opt.momentum = 0.5;
opt.nEpochs = 500;
opt.schedule = [0.3, 0.6, 0.9];
====================================================================
dataset version 11
====================================================================
epoch:
acc:
tr_acc:
opt.batchSize = 64;
opt.LR = 0.1;
opt.weightDecay = 5e-4;
opt.momentum = 0.09;
opt.nEpochs = 1600;
opt.schedule = [0.3, 0.6, 0.9];
opt.warmup = 10;
"""

' recording training settings and results\n===================================\nweight-pruning-ratio:\nfinal acc:93.03\nepoch:539\nopt.batchSize = 32;\nopt.LR = 0.05;\nopt.weightDecay = 5e-3;\nopt.momentum = 0.04;\nopt.nEpochs = 1000;\nopt.schedule = [0.15, 0.3, 0.45];\nopt.warmup = 10;\n====================================================================\n###針對二次剪枝後的重訓練，這個設定滿有效果的，要實驗是哪個參數造成的效果。\nepoch:293\nacc:89.4\ntr_acc:76.4\nopt.batchSize = 64;\nopt.LR = 0.1;\nopt.weightDecay = 5e-3;\nopt.momentum = 0.5;\nopt.nEpochs = 500;\nopt.schedule = [0.3, 0.6, 0.9];\n====================================================================\ndataset version 11\n====================================================================\nepoch:\nacc:\ntr_acc:\nopt.batchSize = 64;\nopt.LR = 0.1;\nopt.weightDecay = 5e-4;\nopt.momentum = 0.09;\nopt.nEpochs = 1600;\nopt.schedule = [0.3, 0.6, 0.9];\nopt.warmup = 10;\n'

In [32]:
def main():
    map_dict_train = {
        '52':1, #alarm
        '56':2, #moaning
        '71':3,
        '99':4, #other_sounds
    };
    
    opt = getOpts();
    opt.pruningRatio = 0.85
    save_dir = "../../../trained_models/step_5_retrain_after_step_4/multifold/retrain_4C_IterPrun_prunratio{}_{}/".format(opt.pruningRatio * 100,getDateStr());
    if not os.path.exists(save_dir):
        os.mkdir(save_dir)
    print(f"save dir is: {save_dir}");
    opt.current_fold = "notset"
    opt.trainData = "notset";#"../../../../uec_iot_models_datasets/multifold/train/version15_multifold_home_fold1/fold1_train_20240721013720.npz"
    opt.valData = "notset";#"../../../../uec_iot_models_datasets/multifold/val/version15_multifold_home_fold1/final_fold1_val_version15_multifold_home.npz"
    #Leqarning settings
    # opt.first_save_acc = 90.0;
    # opt.save_val_acc = 90.0;
    # opt.save_train_acc = 80.0;
    
    opt.batchSize = 64;
    opt.LR = 0.1;
    opt.weightDecay = 5e-4;
    opt.momentum = 0.9;
    opt.nEpochs = 1600;#1600;
    opt.schedule = [0.3, 0.6, 0.9];
    opt.warmup = 10;
    opt.saveDir = save_dir;
    # opt.device="mps";
    if torch.backends.mps.is_available():
        opt.device="mps"; #for apple m2 gpu
    elif torch.cuda.is_available():
        opt.device="cuda:0"; #for nVidia gpu
    else:
        opt.device="cpu"
    print(f"***Use device:{opt.device}");
    # opt.device = torch.device("cuda:0" if  else "cpu");
    #Basic Net Settings
    opt.nClasses = 4
    opt.nFolds = 1;
    opt.splits = [i for i in range(1, opt.nFolds + 1)];
    opt.sr = 20000;
    opt.inputLength = 20150;
    #Test data
    opt.nCrops = 2;
    opt.model_name = "retrained_4C_ratio{}_val{}_tracc_{}@{}epoch_{}.pt"
    #Starting retraining process
    
    foldlist = ['fold1','fold2','fold3','fold4'];
    train_data_list = [
        "fold1_train_20240721013720.npz",
        "fold2_train_20240721013720.npz",
        "fold3_train_20240721013720.npz",
        "fold4_train_20240721013721.npz"
    ];
    val_data_list = [
        "final_fold1_val_version15_multifold_home.npz",
        "final_fold2_val_version15_multifold_home_20240721023951.npz",
        "final_fold3_val_version15_multifold_home_20240721024246.npz",
        "final_fold4_val_version15_multifold_home_20240721024338.npz"
    ];
    print("start to retrain model after second pruning in other 4-fold datasets");
    for i in range(len(foldlist)):
        opt.current_fold = foldlist[i];
        print(f"=================== Using {foldlist[i] dataset to train.} ===================");
        opt.trainData = "../../../../uec_iot_models_datasets/multifold/train/version15_multifold_home_{}/{}".format(foldlist[i],train_data_list[i]);
        opt.valData = "../../../../uec_iot_models_datasets/multifold/val/version15_multifold_home_{}/{}".format(foldlist[i],val_data_list[i]);
        trainer = ReTrainer(opt=opt, classes_dict=map_dict_train);
        trainer.Train();
        del trainer;

In [33]:
main()

save dir is: ../../../trained_models/step_5_retrain_after_step_4/multifold/retrain_4C_IterPrun_prunratio85.0_2024072316/
***Use device:cuda:0
start to retrain model after second pruning in other 4-fold datasets
length of samples:711
config is [8, 44, 25, 53, 53, 93, 96, 169, 165, 253, 324, 4]
ACDNet model has been prepared for training
+----------------------------------------------------------------------------+
+                           Pytorch Model Summary                            +
------------------------------------------------------------------------------
   Layer (type)       Input Shape      Output Shape    Param #      FLOPS #
       Conv2d-1     (1, 1, 20150)     (8, 1, 10071)         72      725,112
  BatchNorm2d-2     (8, 1, 10071)     (8, 1, 10071)         16            0
         ReLu-3     (8, 1, 10071)     (8, 1, 10071)          0       80,568
       Conv2d-4     (8, 1, 10071)     (44, 1, 5034)      1,760    8,859,840
  BatchNorm2d-5     (44, 1, 5034)     (44, 1,